<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Pre-requisites" data-toc-modified-id="Pre-requisites-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pre-requisites</a></span></li><li><span><a href="#Instructions" data-toc-modified-id="Instructions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Instructions</a></span></li><li><span><a href="#Importss" data-toc-modified-id="Imports-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Imports </a></span></li><li><span><a href="#Export-Images" data-toc-modified-id="Export-Images-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Export Images</a></span></li></ul></div>

## Pre-requisites
Register a Google account at [https://code.earthengine.google.com](https://code.earthengine.google.com). This process may take a couple of days. Without registration, the `ee.Initialize()` command below will throw an error message.

## Instructions

This notebook exports VIIRS-DNB  nightlights data from Google Earth Engine.

The images are saved in Geotiff format. By default, this notebook exports images to Google Drive. If you instead prefer to export images to Google Cloud Storage (GCS), change the `EXPORT` constant below to `'gcs'` and set `BUCKET` to the desired GCS bucket name.

Our tasks in this exercise:

Get a yearly VIIRS-DNB composite and clip to a Geometry (a buffer around a point)

## Imports and Constants

In [1]:
import ee
import geemap

In [2]:
from google.colab import drive
drive.mount('/content/google_drive')

Drive already mounted at /content/google_drive; to attempt to forcibly remount, call drive.mount("/content/google_drive", force_remount=True).


Before using the Earth Engine API, you must perform a one-time authentication that authorizes access to Earth Engine on behalf of your Google account you registered at [https://code.earthengine.google.com](https://code.earthengine.google.com). The authentication process saves a credentials file to `$HOME/.config/earthengine/credentials` for future use.

The following command `ee.Authenticate()` runs the authentication process. Once you successfully authenticate, you may comment out this command because you should not need to authenticate again in the future, unless you delete the credentials file. If you do not authenticate, the subsequent `ee.Initialize()` command below will fail.

For more information, see [https://developers.google.com/earth-engine/python_install-conda.html](https://developers.google.com/earth-engine/python_install-conda.html).

In [4]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ALnY5Fxvd-bH16HCFYVR5WfAHQeYQGNH7Q1DZVwJHWA&tc=tgpW67JKf7qw66vBGyiXUS98chwsr-pFigyvMC9q5uk&cc=zbOuZNodlwiXxEsRMvAikP0SBVc6CKJ9Xa89779sE8o

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiIf658k6CfUUiFErDnApRejEkd6zPqjBHBUtaWYZG4Kf_x0a4zMe0

Successfully saved authorization token.


In [5]:
ee.Initialize()  # initialize the Earth Engine API

In [7]:
# get 2018 image, we're using the "avg_rad" band
viirs2018_01 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate('2018-01-01','2018-12-31').select('avg_rad').mean()


# initialize our map
map1 = geemap.Map()
map1.add_basemap('SATELLITE')
map1.addLayer(viirs2018_01, {}, "VIIRS-DNB 2018")

map1.addLayerControl()
map1

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [8]:
lat = 27.71
lon = 85.32
aoi = ee.Geometry.Point([lon, lat]).buffer(200000);

In [9]:
viirs2018_01_clipped = viirs2018_01.clip(aoi)

map2 = geemap.Map(center=[lat, lon],zoom=7)
map2.add_basemap('SATELLITE')
map2.addLayer(viirs2018_01_clipped, {}, "VIIRS-DNB- ktm 2018")
map2.addLayerControl()
map2

Map(center=[27.71, 85.32], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

## Exports

In [3]:
task = ee.batch.Export.image.toDrive(image=viirs2018_01_clipped,
                                     scale=1000,
                                     fileFormat='GeoTIFF',
                                     description='v2ktm_VIIRS_2018',
                                     folder='tmp',
                                     maxPixels=1e10)

task.start()